## 과정

1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 > 벡터 DB 에 저장
4. 쿼리 > 벡터 DB 에 유사도 검색
5. 유사도 검색으로 가져운 문서를 LLM에 답변과 같이 응답

## 생각

음 spotify api 문서들 쭉 파싱해볼까나

## packages

```sh
uv add langchain-community beautifulsoup4 langchain-text-splitters langchain-chroma langchain langchainhub
```

## Docs

- [LangChain Document loaders Docs](https://docs.langchain.com/oss/python/integrations/document_loaders)


In [1]:
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup
from bs4 import XMLParsedAsHTMLWarning
import warnings

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

def simple_extractor(html):
    soup = Soup(html, "html.parser")
    # 불필요한 요소(네비게이션, 헤더, 푸터 등) 제거
    for script in soup(["script", "style", "nav", "footer"]):
        script.decompose()
    return soup.get_text(separator="\n", strip=True)

url = "https://developer.spotify.com"

loader = RecursiveUrlLoader(
    url=url,
    max_depth=5,  # 탐색할 깊이 설정 (너무 깊으면 관련 없는 페이지까지 긁을 수 있음)
    extractor=simple_extractor,  # HTML -> 텍스트 변환 로직
    # exclude_dirs=["https://api.example.com/docs/v1/deprecated"], # 제외할 경로
    # timeout=10,
)

docs = loader.load()
print(f"Loaded {len(docs)} documents.")

Loaded 149 documents.


In [2]:
titles = [doc.metadata.get("title", "No Title") for doc in docs]
titles

['Home | Spotify for Developers',
 'Spotify Open Access | Spotify for Developers',
 'iOS SDK | Spotify for Developers',
 'Getting Started with iOS SDK | Spotify for Developers',
 'Spotify Developer Policy | Spotify for Developers',
 'Community | Spotify for Developers',
 'Home | Spotify for Developers',
 'iOS SDK Reference | Spotify for Developers',
 'Spotify Android SDK | Spotify for Developers',
 'Token Swap and Refresh | Spotify for Developers',
 'Spotify Developer Terms | Spotify for Developers',
 'Commercial Hardware | Spotify for Developers',
 'Third Party Licenses | Spotify for Developers',
 'Making Remote Calls | Spotify for Developers',
 '<![CDATA[Spotify for Developers]]>',
 'Web Playback SDK | Spotify for Developers',
 'Application Lifecycle | Spotify for Developers',
 'Embeds | Spotify for Developers',
 'Advanced User Authentication | Spotify for Developers',
 'Web API | Spotify for Developers',
 'iOS Content Linking | Spotify for Developers',
 'Compliance Tips | Spotify fo

In [3]:
docs[27].page_content

'Scopes | Spotify for Developers\nSkip to content\nWeb API\nOverview\nGetting started\nConcepts\nConcepts\nConcepts\nAccess Token\nAPI calls\nApps\nAuthorization\nRedirect URIs\nPlaylists\nQuota modes\nRate limits\nScopes\nSpotify URIs and IDs\nTrack Relinking\nTutorials\nTutorials\nTutorials\nAuthorization code\nAuthorization code PKCE\nClient credentials\nImplicit grant [Deprecated]\nRefreshing tokens\nMigration: Implicit grant to Authorization code\nMigration: Insecure redirect URI\nHow-Tos\nHow-Tos\nHow-Tos\nDisplay your Spotify profile data in a web app\nReference\nAlbums\nAlbums\nAlbums\nGet Album\nGet Several Albums\nGet Album Tracks\nGet User\'s Saved Albums\nSave Albums for Current User\nRemove Users\' Saved Albums\nCheck User\'s Saved Albums\nGet New Releases\nArtists\nArtists\nArtists\nGet Artist\nGet Several Artists\nGet Artist\'s Albums\nGet Artist\'s Top Tracks\nGet Artist\'s Related Artists\nAudiobooks\nAudiobooks\nAudiobooks\nGet an Audiobook\nGet Several Audiobooks\nGe

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200, # 왜 겹치게? 우리가 원하는 문서를 가져오게 할 확률 증가
)

splitted_docs = loader.load_and_split(text_splitter=text_splitter)
len(splitted_docs)

1026

In [5]:
splitted_docs[:5]

[Document(metadata={'source': 'https://developer.spotify.com', 'content_type': 'text/html; charset=utf-8', 'title': 'Home | Spotify for Developers', 'language': 'en'}, page_content='Home | Spotify for Developers\nSkip to content\nBuild with Spotify’s 100 million songs,\n5 million podcasts and much more\nSee it in action\nBuild with Spotify’s 100 million songs,\n5 million podcasts and much more\nSee it in action\nCode tutorial\n1\nGet your top 5 tracks\n2\nSave the 5 songs in a playlist\n3\nListen to the songs right here right now'),
 Document(metadata={'source': 'https://developer.spotify.com/documentation/open-access', 'content_type': 'text/html; charset=utf-8', 'title': 'Spotify Open Access | Spotify for Developers', 'description': 'Distribute your paid content on Spotify, accessible only to your subscriber base', 'language': 'en'}, page_content="Spotify Open Access | Spotify for Developers\nSkip to content\nOpen Access\nOverview\nConcepts\nTutorials\nTutorials\nTutorials\nAccount Li

In [5]:
from langchain_ollama import OllamaEmbeddings

gemma_embeddings = OllamaEmbeddings(model="embeddinggemma:300m")

In [6]:
from langchain_chroma import Chroma

# 문서들로 바로 임베딩 생성 + 데이터베이스 저장
# chroma_db = Chroma.from_documents(
#     documents=splitted_docs,
#     embedding=gemma_embeddings,
#     persist_directory="./chroma_db",
#     collection_name="spotify-api-docs",
# )

# 기존 파일 데이터베이스 불러오기
chroma_db = Chroma(
    collection_name="spotify-api-docs",
    embedding_function=gemma_embeddings,
    persist_directory="./chroma_db",
)

In [8]:
query = "Spotify API 에서 Rate Limit 에 대해 알려주시겠어요?"
retrieved_docs = chroma_db.similarity_search(query)
retrieved_docs

[Document(id='35e0021d-e581-4758-b61d-0e655f8096ed', metadata={'title': 'Rate Limits | Spotify for Developers', 'language': 'en', 'source': 'https://developer.spotify.com/documentation/web-api/concepts/rate-limits', 'content_type': 'text/html; charset=utf-8'}, page_content="API-wide rate limit. A few API endpoints, like the playlist image upload\nendpoint, have a custom rate limit that may differ from your app-wide rate\nlimit. See the body of your API response from Spotify for more information\nabout the error that you have received.\nBuilding your app with rate limits in mind\nEvery app is different and you'll want to plan your app architecture and user\nexperience with rate limits in mind. Here are a few techniques that can help\nyou design an app that works well with Spotify's Web API rate limits:\nApply for extended quota mode\nIf your app is meant to be used by many Spotify users at the same time then you\nshould apply for\nextended quota\nmode\n.\nApps in this mode have a rate l

In [3]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = 'gemma3n:e4b')

In [10]:
prompt = f"""[Identity]
- 당신은 spotify api 개발을 주도한 전문가입니다.
- [Context] 를 참고해서 사용자의 [Question] 에 대해 답변해주세요.

[Context]
{retrieved_docs}

[Question]
{query}
"""

ai_message = llm.invoke(prompt)

In [11]:
from IPython.display import Markdown, display

display(Markdown(ai_message.content))

Spotify API의 Rate Limit에 대해 궁금하시군요. 자세히 설명해 드리겠습니다.

**API-wide Rate Limit:**

*   Spotify API는 앱당 전체에 적용되는 Rate Limit을 가지고 있습니다.
*   일부 API 엔드포인트(예: 플레이리스트 이미지 업로드)는 앱 전체 Rate Limit과 다른 사용자 정의 Rate Limit을 가질 수 있습니다.
*   API 응답 본문에 오류 정보가 포함되어 있으므로, 오류 내용을 확인하여 Rate Limit 관련 정보를 파악하는 것이 중요합니다.

**Rate Limit 관리 전략:**

*   **Extended Quota Mode 신청:** 앱이 많은 사용자에 의해 사용될 것으로 예상된다면 Extended Quota Mode를 신청할 수 있습니다. 이 모드에서는 개발 모드보다 훨씬 높은 Rate Limit을 제공합니다. 개발자 대시보드에서 Request Extension 링크를 통해 신청할 수 있습니다.
*   **Backoff-Retry 전략 구현:** API 호출 시 429 오류가 발생하면, Spotify에서 제공하는 `Retry-After` 헤더에 지정된 시간만큼 대기한 후 다시 시도하는 Backoff-Retry 전략을 사용해야 합니다.
*   **Batch API 활용:** Get Multiple Albums와 같이 여러 작업을 한 번에 처리할 수 있는 Batch API를 활용하여 API 호출 횟수를 줄일 수 있습니다.

**Rate Limit의 중요성:**

*   Rate Limit은 API의 안정성을 유지하고, 제3자 개발자들이 API를 책임감 있게 사용하도록 돕기 위해 존재합니다.
*   Rate Limit을 초과하면 429 오류가 발생하고, 사용자에게 예상치 못한 문제가 발생할 수 있습니다.

**참고 자료:**

*   [Rate Limits | Spotify for Developers](https://developer.spotify.com/documentation/web-api/concepts/rate-limits)

궁금한 점이 있다면 언제든지 다시 질문해주세요.

In [12]:
from langchainhub import Client

client = Client()
prompt = client.pull("rlm/rag-prompt")
prompt

/var/folders/hc/l5s1x0nj069bzjc37bk47xnw0000gn/T/ipykernel_74780/2309490202.py:4: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  prompt = client.pull("rlm/rag-prompt")
/Users/gwagdaehun/dev/personal/ai-playground/rag-scratch/.venv/lib/python3.13/site-packages/langchainhub/client.py:326: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = self.pull_repo(owner_repo_commit)


'{"id": ["langchain", "prompts", "chat", "ChatPromptTemplate"], "lc": 1, "type": "constructor", "kwargs": {"messages": [{"id": ["langchain", "prompts", "chat", "HumanMessagePromptTemplate"], "lc": 1, "type": "constructor", "kwargs": {"prompt": {"id": ["langchain", "prompts", "prompt", "PromptTemplate"], "lc": 1, "type": "constructor", "kwargs": {"template": "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\\nQuestion: {question} \\nContext: {context} \\nAnswer:", "input_variables": ["question", "context"], "template_format": "f-string"}}}}], "input_variables": ["question", "context"]}}'

In [2]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# RetrievalQA deprecated 됨
# 다른걸로 시도해보려 했지만 잘 안됨

from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# 1. 검색된 문서 + 질문 -> 답변 생성 체인
# prompt는 langsmith에서 가져온 'rlm/rag-prompt' 등을 사용
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# 2. 검색기 설정
retriever = chroma_db.as_retriever()

# 3. 최종 RAG 체인 연결
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 4. 실행
result = rag_chain.invoke({
    "input": "Spotify API Rate Limit 정보 알려줘",
    "question": "Spotify API Rate Limit 정보 알려줘"
})
result

{'input': 'Spotify API Rate Limit 정보 알려줘',
 'question': 'Spotify API Rate Limit 정보 알려줘',
 'context': [Document(id='35e0021d-e581-4758-b61d-0e655f8096ed', metadata={'content_type': 'text/html; charset=utf-8', 'source': 'https://developer.spotify.com/documentation/web-api/concepts/rate-limits', 'title': 'Rate Limits | Spotify for Developers', 'language': 'en'}, page_content="API-wide rate limit. A few API endpoints, like the playlist image upload\nendpoint, have a custom rate limit that may differ from your app-wide rate\nlimit. See the body of your API response from Spotify for more information\nabout the error that you have received.\nBuilding your app with rate limits in mind\nEvery app is different and you'll want to plan your app architecture and user\nexperience with rate limits in mind. Here are a few techniques that can help\nyou design an app that works well with Spotify's Web API rate limits:\nApply for extended quota mode\nIf your app is meant to be used by many Spotify users 